IMPORTING ALL THE LIBRARIES NECESSARY:

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Getting all content from the wikipidea page using requests package:

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

response=requests.get(url)

Using BeautifulSoup library to read the html content from the wikipedia page:

In [3]:
soup=BeautifulSoup(response.content,'lxml')


Extracting and Exploring the table content from webpage:

In [4]:
table=soup.find_all('table',class_='wikitable sortable')
table=table[0]
type(table)

bs4.element.Tag

Writing all content of the table in a file named stats.txt in a csv format:

In [31]:
with open ('stats.txt', 'w') as r:
    r.write('0,Postcode,Borough,Neighbourhood')
    r.write('\n')
    for row in table.find_all('tr'):
        for cell in row.find_all('td'):
            r.write(',')
            r.write(cell.text)
            
        

Reading the stats.txt file as a csv file import it as a dataframe using Pandas library:

In [32]:
df=pd.read_csv('stats.txt')

PRE-PROCESSING THE DATAFRAME:

In [33]:
df.drop('0',axis=1,inplace=True)
df=df[df['Borough']!='Not assigned']
df.reset_index(drop=True,inplace=True)

In [34]:
df.head(5)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


If more than one neighborhood existed in one postal code area (like in row 2 and 3 having Postcode='M5A', then merging the two rows:

In [38]:
for i in range(0,400):
    if i<(df.shape[0]-1):
        if df.loc[i,'Postcode']==df.loc[i+1,'Postcode']:
            df.loc[i,'Neighbourhood']=df.loc[i,'Neighbourhood']+', '+df.loc[i+1,'Neighbourhood']
            df.drop(i+1,axis=0,inplace=True)
            df.reset_index(drop=True,inplace=True)

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough:

In [39]:
for i in range(0,df.shape[0]):
    if df.loc[i,'Neighbourhood']=='Not assigned':
        df.loc[i,'Neighbourhood']=df.loc[i,'Borough']

In [40]:
df.shape

(103, 3)

Creating two more columns for Latitude and Longitude:

In [41]:
df['latitude']=df.index
df['longitude']=df.index
df.head()

,Postcode,Borough,Neighbourhood,latitude,longitude
0,M3A,North York,Parkwoods,0,0
1,M4A,North York,Victoria Village,1,1
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",2,2
3,M6A,North York,"Lawrence Heights, Lawrence Manor",3,3
4,M7A,Queen's Park,Queen's Park,4,4


I used the Geospatial_Coordinates.csv file provided to extract the latittude and longitude of different neighbourhoods:

In [42]:
coord=pd.read_csv('Geospatial_Coordinates.csv')

In [43]:
coord.columns=['Postcode', 'Latitude', 'Longitude']
coord.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Filling the correct values of latitude and longitude in main dataframe i.e. df:

In [44]:
for i in range(0,df.shape[0]):
    for j in range(0,coord.shape[0]):
        if df.loc[i,'Postcode']==coord.loc[j,'Postcode']:
            df.loc[i,'latitude']=coord.loc[j,'Latitude']
            df.loc[i,'longitude']=coord.loc[j,'Longitude']

In [46]:
df.head()

,Postcode,Borough,Neighbourhood,latitude,longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [47]:
print('The dataframe has {} different boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 11 different boroughs and 103 neighborhoods.


Storing the final dataframe in a CSV file named data1.csv in correct format:

In [53]:
df.index=df['Postcode']
df.drop('Postcode',axis=1,inplace=True)

df.to_csv('data1.csv')